In [1]:
#!pip install tensorflow_hub
#!pip install tensorflow_text
#!pip install numpy
#!pip install tensorflow
#!pip install pandas
#!pip install SenticNet
#!pip install tensorflow.keras 

In [2]:
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import tensorflow as tf
import pandas as pd
# from senticnet.senticnet import SenticNet
import re
import string
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
#github test first commit

In [3]:
# disable GPU
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [6]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_model = hub.KerasLayer(encoder_url)

In [7]:
# input : 126 words as sentence  if exceeds 126, split it, if less than 126 apply padding of 0.
# output : 128 * 768

def bert_process(text_to_process):
    text_preprocessed = bert_preprocess_model(text_to_process)

    inputk = {'input_word_ids' : [], 'input_mask': [], 'input_type_ids': []}
    
    for i in text_preprocessed.keys():
        index = 0

        for arr in text_preprocessed[i]:
            arr = list(arr.numpy())
            arr = list(filter(lambda num: num != 0, arr))
            if len(inputk[i]) == 0:
                inputk[i].append(arr)
            else:
                inputk[i][index] = list(filter(lambda num: num != 0, inputk[i][index]))
                if len(inputk[i][index]) + len(arr) -1 > 128:
                    inputk[i].append(arr)
                    lengthwithoutzero = len(inputk[i][index])
                    padd = [0]*(128-lengthwithoutzero)
                    inputk[i][index].extend(padd) 
                    index += 1
                else:
                    inputk[i][index].extend(arr[1:])             
        lengthwithoutzero = len(inputk[i][index])
        padd = [0]*(128-lengthwithoutzero)
        inputk[i][index].extend(padd)

    if len(inputk["input_word_ids"]) < 6:
        paddinput = [0]*128
        for arr in range((6-len(inputk["input_word_ids"]))):
            inputk["input_word_ids"].append(paddinput)
            inputk["input_mask"].append(paddinput)

            
    inputk["input_mask"] = tf.convert_to_tensor(inputk["input_mask"])
    inputk["input_type_ids"] = tf.convert_to_tensor(inputk["input_type_ids"])
    inputk["input_word_ids"] = tf.convert_to_tensor(inputk["input_word_ids"])

    bert_results = bert_model(inputk)
    tempres = (bert_results["sequence_output"].numpy()).tolist()
    
    return tempres

In [8]:
def clean_text(text):
    regex = re.compile('[%s]' % re.escape('|'))
    text = regex.sub(" ", text)
    words = str(text).split()
    words = [i.lower() + " " for i in words]
    words = [i for i in words if not "http" in i]
    words = " ".join(words)
    # words = words.translate(words.maketrans('', '', string.punctuation))
    return words

In [9]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|watch)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("|||","<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [10]:
MBTI = pd.read_csv('mbti_1.csv')
data = []

In [11]:
# sample is number of samples being used.
# total data = 
# [sample, 6, 128, 768]

# sample = 1000

# for n in range(1,int(sample/100) + 1):
#     for post in MBTI["posts"][100*(n-1):(100 * n)]:
#         sentence = clean_text(str(post))
#         sentence = split_into_sentences(sentence)
#         if len(sentence) < 50:
#             temp = [""] * (50-len(sentence))
#             sentence = sentence + temp
#         processedinput = bert_process(sentence)
#         data.append(processedinput[:6])


# for n in range(3, 11):
#     for post in MBTI["posts"][100*(n-1):(100 * n)]:
#         sentence = clean_text(str(post))
#         sentence = split_into_sentences(sentence)
#         processedinput = bert_process(sentence)
#         data.append(processedinput[:6])

labels = []
templabels = MBTI["type"]
labelmap = {"I" : 0.0, "E": 1.0, "N": 0.0, "S" : 1.0, "F": 0.0, "T": 1.0, "J": 0.0, "P": 1.0}


for l in templabels:
    labels.append([labelmap[l[0]], labelmap[l[1]], labelmap[l[2]], labelmap[l[3]]])

Y_label = []

# for n in range(1, 2):
#     for post in MBTI["posts"][100*(n-1):(100 * n)]:
#         sentence = clean_text(str(post))
#         sentence = split_into_sentences(sentence)
#         processedinput = bert_process(sentence)
#         data.append(processedinput[:6])

k=0

for n in range(5*7+1, 5*(7+1)+1):
    for idx in range(100*(n-1),(100 * n)):
        post = MBTI["posts"][idx]
        sentence = clean_text(str(post))
        sentence = split_into_sentences(sentence)
        if len(sentence) != 0:
            processedinput = bert_process(sentence)
            data.append(processedinput[:14])
            del processedinput
            Y_label.append(labels[idx])

In [12]:
print(len(data))
print(len(data[0]))
print(len(processedinput[0]))
print(len(processedinput[0][0]))

100
6
128
768


In [13]:
# print(len(data))
# for i in data:
#     if len(i) != 6:
#         print(len(i))

In [14]:
'''
# labels = []
IElabels = []
NSlabels = []
FTlabels = []
JPlabels = []

templabels = MBTI["type"]
# labelmap = {"INTJ" : 0.0, "INTP": 1.0, "ENTJ": 2.0, "ENTP" : 3.0, "INFJ": 4.0, "INFP": 5.0, "ENFJ": 6.0, "ENFP": 7.0, "ISTJ": 8.0, "ISFJ": 9.0, "ESTJ": 10.0, "ESFJ": 11.0, "ISTP": 12.0, "ISFP": 13.0, "ESTP": 14.0, "ESFP": 15.0}
# labeindex = ["INTJ", "INTP", "ENTJ", "ENTP", "INFJ", "INFP", "ENFJ", "ENFP", "ISTJ", "ISFJ", "ESTJ", "ESFJ", "ISTP", "ISFP", "ESTP", "ESFP"]
labelmap = {"I" : 0.0, "E": 1.0, "N": 0.0, "S" : 1.0, "F": 0.0, "T": 1.0, "J": 0.0, "P": 1.0}

for l in templabels:
    IElabels.append(labelmap[l[0]])
    NSlabels.append(labelmap[l[1]])
    FTlabels.append(labelmap[l[2]])
    JPlabels.append(labelmap[l[3]])
'''

'\n# labels = []\nIElabels = []\nNSlabels = []\nFTlabels = []\nJPlabels = []\n\ntemplabels = MBTI["type"]\n# labelmap = {"INTJ" : 0.0, "INTP": 1.0, "ENTJ": 2.0, "ENTP" : 3.0, "INFJ": 4.0, "INFP": 5.0, "ENFJ": 6.0, "ENFP": 7.0, "ISTJ": 8.0, "ISFJ": 9.0, "ESTJ": 10.0, "ESFJ": 11.0, "ISTP": 12.0, "ISFP": 13.0, "ESTP": 14.0, "ESFP": 15.0}\n# labeindex = ["INTJ", "INTP", "ENTJ", "ENTP", "INFJ", "INFP", "ENFJ", "ENFP", "ISTJ", "ISFJ", "ESTJ", "ESFJ", "ISTP", "ISFP", "ESTP", "ESFP"]\nlabelmap = {"I" : 0.0, "E": 1.0, "N": 0.0, "S" : 1.0, "F": 0.0, "T": 1.0, "J": 0.0, "P": 1.0}\n\nfor l in templabels:\n    IElabels.append(labelmap[l[0]])\n    NSlabels.append(labelmap[l[1]])\n    FTlabels.append(labelmap[l[2]])\n    JPlabels.append(labelmap[l[3]])\n'

In [15]:
labels = []
templabels = MBTI["type"]
labelmap = {"I" : 0.0, "E": 1.0, "N": 0.0, "S" : 1.0, "F": 0.0, "T": 1.0, "J": 0.0, "P": 1.0}


for l in templabels:
    labels.append([labelmap[l[0]], labelmap[l[1]], labelmap[l[2]], labelmap[l[3]]])

#split train, val, test data


In [16]:
print(labels[1])

[1.0, 0.0, 1.0, 1.0]


In [21]:
Y_label = labels[:100]

In [22]:
'''
Y_IE_label = IElabels[:sample]
Y_NS_label = NSlabels[:sample]
Y_FT_label = FTlabels[:sample]
Y_JP_label = JPlabels[:sample]
'''

'\nY_IE_label = IElabels[:sample]\nY_NS_label = NSlabels[:sample]\nY_FT_label = FTlabels[:sample]\nY_JP_label = JPlabels[:sample]\n'

In [23]:
# sum of val data and test data
TEST_SIZE = 0.4
# portion of val from test
VAL_SIZE = 0.5

In [24]:
# split IE_label to test train
X_train, X_test, Y_train, Y_test = train_test_split(
    data, Y_label, test_size=TEST_SIZE, random_state=42)

In [25]:
# split IE_label to test val
X_val, X_test, Y_val, Y_test = train_test_split(
    X_test, Y_test, test_size = VAL_SIZE, random_state=42)

In [26]:
# split IE_label to test train
#X_train, X_IEtest, Y_train, Y_IEtest = train_test_split(
#    data, Y_IE_label, test_size=TEST_SIZE, random_state=42)

In [27]:
# split IE_label to test val
#X_IE_val, X_IE_test, Y_IE_val, Y_IE_test = train_test_split(
#    X_IEtest, Y_IEtest, test_size = VAL_SIZE, random_state=42)

In [28]:
# split NS_label to test train
#X_train, X_NStest, Y_train, Y_NStest = train_test_split(
#    data, Y_NS_label, test_size = TEST_SIZE, random_state=42)

In [29]:
# split NS_label to test val
#X_NS_val, X_NS_test, Y_NS_val, Y_NS_test = train_test_split(
#    X_NStest, Y_NStest, test_size = VAL_SIZE, random_state=42)

In [30]:
# split FT_label to test train
#X_train, X_FTtest, Y_train, Y_FTtest = train_test_split(
#    data, Y_FT_label, test_size = TEST_SIZE, random_state=42)

In [31]:
# split FT_label to test val
#X_FT_val, X_FT_test, Y_FT_val, Y_FT_test = train_test_split(
#    X_FTtest, Y_FTtest, test_size = VAL_SIZE, random_state=42)

In [32]:
# split JP_label to test train
#X_train, X_JPtest, Y_train, Y_JPtest = train_test_split(
#    data, Y_JP_label, test_size = TEST_SIZE, random_state=42)

In [33]:
# split JP_label to test val
#X_JP_val, X_JP_test, Y_JP_val, Y_JP_test = train_test_split(
#    X_JPtest, Y_JPtest, test_size = VAL_SIZE, random_state=42)

# IE model

In [34]:
#convert to tensor
X_train = tf.convert_to_tensor(X_train)
Y_train = tf.convert_to_tensor(Y_train)
X_val = tf.convert_to_tensor(X_val)
Y_val = tf.convert_to_tensor(Y_val)
X_test = tf.convert_to_tensor(X_test)
Y_test = tf.convert_to_tensor(Y_test)

In [35]:
# model constructor for CNN classification
def model_fn() :
  model = models.Sequential()
  model.add(layers.Conv2D(32, (1, 3), activation='leaky_relu', input_shape=X_train.shape[1:]))
  model.add(layers.Conv2D(32, (3, 5), activation='leaky_relu', padding = 'same'))
  model.add(layers.Conv2D(32, (5, 5), activation='leaky_relu', padding = 'same'))
  model.add(layers.Conv2D(32, (5, 3), activation='leaky_relu', padding = 'same'))
  model.add(layers.Conv2D(32, (3, 1), activation='leaky_relu', padding = 'same'))

  # dropout
  # model.add(layers.Dropout(0.3))
  # fully connected layer
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='sigmoid'))
  model.add(layers.Dense(4, activation='sigmoid'))
  #
  return model

In [36]:
model = model_fn()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 6, 126, 32)        73760     
                                                                 
 conv2d_1 (Conv2D)           (None, 6, 126, 32)        15392     
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 126, 32)        25632     
                                                                 
 conv2d_3 (Conv2D)           (None, 6, 126, 32)        15392     
                                                                 
 conv2d_4 (Conv2D)           (None, 6, 126, 32)        3104      
                                                                 
 flatten (Flatten)           (None, 24192)             0         
                                                                 
 dense (Dense)               (None, 128)               3

In [37]:
# model = tf.keras.models.load_model('saved_model.h5')


In [38]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


checkpoint_cb = keras.callbacks.ModelCheckpoint('best-model.h5', 
                                                save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=3,
                                                 restore_best_weights=True)

model.fit(X_train, Y_train, epochs=15, batch_size = 32,
                    validation_data=(X_val, Y_val))


Epoch 1/15
2/2 [==============================] - 10s 1s/step - loss: 0.6820 - accuracy: 0.3000 - val_loss: 0.5751 - val_accuracy: 0.1500
Epoch 2/15
2/2 [==============================] - 0s 83ms/step - loss: 0.5239 - accuracy: 0.2667 - val_loss: 0.5908 - val_accuracy: 0.1500
Epoch 3/15
2/2 [==============================] - 0s 115ms/step - loss: 0.4638 - accuracy: 0.2667 - val_loss: 0.5824 - val_accuracy: 0.1500
Epoch 4/15
2/2 [==============================] - 0s 109ms/step - loss: 0.4092 - accuracy: 0.2833 - val_loss: 0.5739 - val_accuracy: 0.1500
Epoch 5/15
2/2 [==============================] - 0s 111ms/step - loss: 0.3344 - accuracy: 0.5500 - val_loss: 0.5757 - val_accuracy: 0.1500
Epoch 6/15
2/2 [==============================] - 0s 143ms/step - loss: 0.2364 - accuracy: 0.5000 - val_loss: 0.7366 - val_accuracy: 0.3500
Epoch 7/15
2/2 [==============================] - 0s 113ms/step - loss: 0.2101 - accuracy: 0.6500 - val_loss: 0.7454 - val_accuracy: 0.1500
Epoch 8/15
2/2 [=======

In [39]:
model.evaluate(X_test, Y_test)

1/1 [==============================] - 0s 56ms/step - loss: 0.9077 - accuracy: 0.1000


[0.9077480435371399, 0.10000000149011612]

In [40]:
model.save("saved_model.h5")

In [41]:
# tf.keras.models.load_model('saved_model.h5')

In [42]:
# data = []
# for n in range(11, 21):
#     for post in MBTI["posts"][100*(n-1):(100 * n)]:
#         sentence = clean_text(str(post))
#         sentence = split_into_sentences(sentence)
#         if len(sentence) < 50:
#             temp = [""] * (50-len(sentence))
#             sentence = sentence + temp
#         processedinput = bert_process(sentence)
#         data.append(processedinput[:6])

In [43]:
# Y_label = labels[1000: 2000]

In [44]:
# X_train, X_test, Y_train, Y_test = train_test_split(
#     data, Y_label, test_size=TEST_SIZE, random_state=42)

In [45]:
# X_val, X_test, Y_val, Y_test = train_test_split(
#     X_test, Y_test, test_size = VAL_SIZE, random_state=42)

In [46]:
# X_train = tf.convert_to_tensor(X_train)
# Y_train = tf.convert_to_tensor(Y_train)
# X_val = tf.convert_to_tensor(X_val)
# Y_val = tf.convert_to_tensor(Y_val)
# X_test = tf.convert_to_tensor(X_test)
# Y_test = tf.convert_to_tensor(Y_test)

In [47]:
# model.fit(X_train, Y_train, epochs=20, batch_size = 32,
#                     validation_data=(X_val, Y_val))
#                     #callbacks=[checkpoint_cb, early_stopping_cb])

In [48]:
# model.evaluate(X_test, Y_test)

In [49]:
# model.save("saved_model.h5")

# Predict


In [50]:
def preprocess_text(text) : 
    sentence = clean_text(str(text))
    sentence = split_into_sentences(sentence)
    if len(sentence) < 50:
        temp = [""] * (50-len(sentence))
        sentence = sentence + temp
    processedinput = bert_process(sentence)
    # input.append(processedinput[:6])
    return processedinput

In [51]:
text1 = "Omg pls Ull be the best honda7 driver ever One day ill see the coolest honda7. Hmm I honestly dont know much about cats Cars* LOL. Like a chaotic one? I think? U know me im p quite. Go to bed. Omg am i finally seeing allen on valo?! Wht the heck Why r u still up. Lollll GO TO BED Wowow so many friends. Ive never been this proud Like a proud mom. Cant wait to get carried by u😍"
text2 = "yoooooo looks so good did it hurt a lot lowkey wanna get mine too OMG thats scary as hell cuz u look so good yeah true but ur nose looks so pretty how much did it cost u what the fuck thats a lot YES PLEASE COME i feel like you got skinnier like u have no fat TEACH ME still under 50 right hell nahhhh thats insane tho its all edits btw"
text3 = "Dang this is good It def needs some practice although we’ve been told how to skim thru papers. I think im getting better at it too I skimmed thru so many papers for my grad apps and lab searches. Also Its 9:44 and i alr had a coffee but it wasnt enough of caffeine for me Should i get another one or am i going too crazy. I hv a quiz tmrw 9am so im not sure if i can sleep today lolll. Well the quiz is alr at early in the morning So cant rly study beforehand. Its more like reviewing Idk im honestly fcked at this point Should hv gone to the lectures at least but they were at 9am and the lecturer is so terrible so .. "

input1 = [preprocess_text(text1)]
input2 = [preprocess_text(text2)]
input3 = [preprocess_text(text3)]


In [52]:
input1 = tf.convert_to_tensor(input1)
prediction1 = model.predict(input1)
print(prediction1[0])

1/1 [==============================] - 0s 388ms/step
[0.34086472 0.55701387 0.21050777 0.68700796]


In [53]:
ielabel = ["I", "E"]
nslabel = ["N","S"]
ftlabel = ["F", "T"]
jplabel = ["J", "P"]

mbti = ielabel[round(prediction1[0][0])] + nslabel[round(prediction1[0][1])] + ftlabel[round(prediction1[0][2])] + jplabel[round(prediction1[0][3])]

print(mbti)

ISFP


In [54]:
input2 = tf.convert_to_tensor(input2)
prediction = model.predict(input2)
print(prediction[0])

1/1 [==============================] - 0s 33ms/step
[0.16191423 0.6875887  0.11615215 0.6795502 ]


In [55]:
ielabel = ["I", "E"]
nslabel = ["N","S"]
ftlabel = ["F", "T"]
jplabel = ["J", "P"]

mbti = ielabel[round(prediction[0][0])] + nslabel[round(prediction[0][1])] + ftlabel[round(prediction[0][2])] + jplabel[round(prediction[0][3])]

print(mbti)

ISFP


In [56]:
input3 = tf.convert_to_tensor(input3)
prediction = model.predict(input3)
print(prediction[0])

1/1 [==============================] - 0s 34ms/step
[0.34161758 0.22485428 0.1376398  0.7524449 ]


In [57]:
ielabel = ["I", "E"]
nslabel = ["N","S"]
ftlabel = ["F", "T"]
jplabel = ["J", "P"]

mbti = ielabel[round(prediction[0][0])] + nslabel[round(prediction[0][1])] + ftlabel[round(prediction[0][2])] + jplabel[round(prediction[0][3])]

print(mbti)

INFP


In [58]:
text = "Lol. Hangover is so painful. Do u know if theyre giving out the tests today? 329? Hehe i know~. It was so hard to wake up tho. I had like 40 alarms just to wake up at 9 ;). Okie then i’ll see u in class!. omg so cutee. And thankk uu. Do u know how to do this question?? Why??? Wut about this. Thank uuu. Ahahaha. Okiee. Ohhh okayy. Okieeee thank uu. Yeayea. Im free! Just give me 10 minutes. Yupyup let me jnow when ur donw."
input = [preprocess_text(text)]
input = tf.convert_to_tensor(input)
prediction = model.predict(input)
print(prediction[0])

1/1 [==============================] - 0s 29ms/step
[0.5673296  0.5036518  0.12522899 0.80509   ]


In [59]:
ielabel = ["I", "E"]
nslabel = ["N","S"]
ftlabel = ["F", "T"]
jplabel = ["J", "P"]

mbti = ielabel[round(prediction[0][0])] + nslabel[round(prediction[0][1])] + ftlabel[round(prediction[0][2])] + jplabel[round(prediction[0][3])]

print(mbti)

ESFP
